In [93]:
! pip install tensorflow scikit-learn torch torchvision pennylane umap-learn imbalanced-learn

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import DataLoader,Dataset
import pennylane as qml
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import umap
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import joblib
import random

In [3]:
# ---------------------- LOAD LABELS ----------------------
labels_df = pd.read_csv("experiment-ii_labels.csv")
base_dir = "experiment-ii"
file_paths, unique_labels = [], []

for _, row in labels_df.iterrows():
    full_path = os.path.join(base_dir, row["Subject"], row["Mat_Type"], row["File_Name"])
    if os.path.exists(full_path):
        file_paths.append(full_path)
        unique_labels.append(row["Label"])

label_mapping = {label: idx for idx, label in enumerate(sorted(set(unique_labels)))}
labels = [label_mapping[lbl] for lbl in unique_labels]

# ---------------------- LOAD MATRIX FILES FROM .TXT ----------------------
X = []
for path in file_paths:
    try:
        matrix = np.loadtxt(path)  # Assumes whitespace- or tab-delimited .txt files
        X.append(matrix.flatten())  # Flatten 2D matrix to 1D
    except Exception as e:
        print(f"Failed to load {path}: {e}")

X = np.array(X)
Y = np.array(labels)

In [4]:
# 1. Preprocessing
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# 2. Autoencoder for initial dimensionality reduction
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, bottleneck_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, bottleneck_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        return self.decoder(encoded)

    def encode(self, x):
        return self.encoder(x)

input_dim = X_scaled.shape[1]
bottleneck_dim = 50
autoencoder = AutoEncoder(input_dim, bottleneck_dim)
criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

In [6]:
# Train Autoencoder
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
for epoch in range(150):
    optimizer.zero_grad()
    output = autoencoder(X_tensor)
    loss = criterion(output, X_tensor)
    loss.backward()
    optimizer.step()

X_encoded = autoencoder.encode(X_tensor).detach().numpy()

In [7]:
# 3. Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy='auto', k_neighbors=3, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_encoded, Y)


# Print class distribution after SMOTE
label_counts = Counter(y_resampled)
print("Class distribution after SMOTE:")
for label, count in label_counts.items():
    print(f"Label {label}: {count} samples")

Class distribution after SMOTE:
Label 2: 271 samples
Label 1: 271 samples
Label 0: 271 samples


In [8]:
# 3. UMAP for further dimensionality reduction (to match qubit count)
n_qubits = 11
umap_model = umap.UMAP(n_components=n_qubits, random_state=42)
X_umap = umap_model.fit_transform(X_resampled)

import joblib
joblib.dump(umap_model, "umap4_model.pkl")

d:\QCNN\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
d:\QCNN\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


['umap4_model.pkl']

In [9]:
# 5. Train/Test Split AFTER UMAP + SMOTE
X_train, X_test, y_train, y_test = train_test_split(
    X_umap, y_resampled, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_resampled
)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

batch_size = 16

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"🔹 Number of training samples: {len(train_loader.dataset)}")
print(f"🔹 Number of testing samples: {len(test_loader.dataset)}")


🔹 Number of training samples: 650
🔹 Number of testing samples: 163


In [10]:
def QuantumConvLayer(inputs, weights):
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev, interface="torch")
    def circuit(x, w):
        qml.AngleEmbedding(x, wires=range(num_qubits), rotation='Y')

        for l in range(num_layers):
            for i in range(num_qubits):
                qml.RY(w[l, i, 0], wires=i)
                qml.RZ(w[l, i, 1], wires=i)

            # Entanglement: First ring
            for i in range(num_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            qml.CNOT(wires=[num_qubits - 1, 0])  # close the ring

            # Optional: Add another layer for deeper entanglement
            for i in range(num_qubits):
                qml.CRX(w[l, i, 0], wires=[i, (i + 2) % num_qubits])

        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    return circuit(inputs, weights)


In [ ]:
def QuantumPoolingLayer(inputs):
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev, interface="torch")
    def circuit(x):
        qml.AngleEmbedding(x, wires=range(num_qubits), rotation='Y')

        # Compress even+odd wires together
        for i in range(0, num_qubits - 1, 2):
            qml.CRY(np.pi / 2, wires=[i, i + 1])
            qml.CZ(wires=[i, i + 1])

        # Interference via H and S gates
        for i in range(num_qubits):
            qml.Hadamard(wires=i)
            qml.S(wires=i)

        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    return circuit(inputs)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QCNN(nn.Module):
    def __init__(self, num_qubits=11, num_layers=4, num_classes=3):
        super(QCNN, self).__init__()
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.q_params = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits, 2))

        # Classical Deep Head: Fully connected layers with normalization and dropout
        self.fc1 = nn.Linear(num_qubits, 256)
        self.norm1 = nn.LayerNorm(256)
        self.fc2 = nn.Linear(256, 128)
        self.norm2 = nn.LayerNorm(128)
        self.fc3 = nn.Linear(128, 64)
        self.norm3 = nn.LayerNorm(64)
        self.fc4 = nn.Linear(64, 32)
        self.norm4 = nn.LayerNorm(32)
        self.fc5 = nn.Linear(32, 16)
        self.out = nn.Linear(16, num_classes)

    def forward(self, x, return_quantum_features=False):
        x = x.view(x.shape[0], -1)
        quantum_outputs = []

        for sample in x:
            conv_out = QuantumConvLayer(sample, self.q_params)
            pool_out = QuantumPoolingLayer(torch.tensor(conv_out, dtype=torch.float32))
            quantum_outputs.append(torch.tensor(pool_out, dtype=torch.float32))

        x = torch.stack(quantum_outputs).to(x.device)

        if return_quantum_features:
            logits = self.out(self.fc5(self.fc4(self.fc3(self.fc2(self.fc1(x))))))
            return logits, x

        x = F.relu(self.norm1(self.fc1(x)))
        x = F.dropout(x, p=0.3)
        x = F.relu(self.norm2(self.fc2(x)))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.norm3(self.fc3(x)))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.norm4(self.fc4(x)))
        x = F.dropout(x, p=0.1)
        x = F.relu(self.fc5(x))
        return self.out(x)


In [14]:
# 6. Initialize
num_layers=4
num_qubits=11
num_classes=3
model = QCNN(num_qubits,num_layers,num_classes)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 1.0, 1.0]))  # Update with true class weights
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.6)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔍 Model Summary (Quantum-Classical Hybrid):\n")
for name, param in model.named_parameters():
    print(f"{name:40} | shape: {tuple(param.shape)} | requires_grad: {param.requires_grad}")


🔍 Model Summary (Quantum-Classical Hybrid):

q_params                                 | shape: (4, 11, 2) | requires_grad: True
fc1.weight                               | shape: (256, 11) | requires_grad: True
fc1.bias                                 | shape: (256,) | requires_grad: True
norm1.weight                             | shape: (256,) | requires_grad: True
norm1.bias                               | shape: (256,) | requires_grad: True
fc2.weight                               | shape: (128, 256) | requires_grad: True
fc2.bias                                 | shape: (128,) | requires_grad: True
norm2.weight                             | shape: (128,) | requires_grad: True
norm2.bias                               | shape: (128,) | requires_grad: True
fc3.weight                               | shape: (64, 128) | requires_grad: True
fc3.bias                                 | shape: (64,) | requires_grad: True
norm3.weight                             | shape: (64,) | requires_grad: 

In [15]:
# Trackers
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

n_epochs = 30

for epoch in range(n_epochs):
    # ---------- TRAINING ----------
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = train_loss / total_train
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # ---------- VALIDATION ----------
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / total_val
    val_accuracy = correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # ---------- LOG ----------
    print(f"Epoch [{epoch+1}/{n_epochs}]")
    print(f"  Train    -> Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy*100:.2f}%")
    print(f"  Validate -> Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy*100:.2f}%\n")


Epoch [1/30]
  Train    -> Loss: 1.1039, Accuracy: 32.77%
  Validate -> Loss: 1.0994, Accuracy: 30.67%

Epoch [2/30]
  Train    -> Loss: 1.1027, Accuracy: 32.92%
  Validate -> Loss: 1.1006, Accuracy: 31.90%

Epoch [3/30]
  Train    -> Loss: 1.0988, Accuracy: 34.46%
  Validate -> Loss: 1.0991, Accuracy: 34.36%

Epoch [4/30]
  Train    -> Loss: 1.1000, Accuracy: 31.54%
  Validate -> Loss: 1.0958, Accuracy: 39.26%

Epoch [5/30]
  Train    -> Loss: 1.1003, Accuracy: 31.23%
  Validate -> Loss: 1.0982, Accuracy: 33.13%

Epoch [6/30]
  Train    -> Loss: 1.0982, Accuracy: 35.85%
  Validate -> Loss: 1.0977, Accuracy: 38.65%

Epoch [7/30]
  Train    -> Loss: 1.1004, Accuracy: 30.46%
  Validate -> Loss: 1.0976, Accuracy: 34.97%

Epoch [8/30]
  Train    -> Loss: 1.0987, Accuracy: 33.54%
  Validate -> Loss: 1.0967, Accuracy: 36.81%

Epoch [9/30]
  Train    -> Loss: 1.0991, Accuracy: 33.23%
  Validate -> Loss: 1.0958, Accuracy: 39.26%

Epoch [10/30]
  Train    -> Loss: 1.0965, Accuracy: 35.69%
  Val

In [16]:
def evaluate_on_test_set(model, test_loader, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            outputs = model(xb)
            loss = criterion(outputs, yb)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == yb).sum().item()
            total += yb.size(0)

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = correct / total

    print("\n--- Final Test Evaluation ---")
    print(f"Test Loss: {avg_test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")

    return avg_test_loss, test_accuracy

# Usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
final_test_loss, final_test_acc = evaluate_on_test_set(model, test_loader, device)



--- Final Test Evaluation ---
Test Loss: 0.8354
Test Accuracy: 57.67%


In [17]:
#------Save Model------------
torch.save(model.state_dict(),"qcnn4_model.pth")